In [40]:
from agents import SQLiteSession

session = SQLiteSession("user_123", "src/openai_study_mode_clone/study_agent_session.db")

msg = await session.get_items()

In [41]:
print(msg)

[{'content': 'i want to lean python', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "Awesome! Python is a great choice.\n\nTo get started, what's your experience with programming? Have you coded before, or is this your first time?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'hy', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': "Got it. Since you're new to coding, we'll start with the basics. Here’s the plan:\n\n1.  **What is Python?** Briefly, what it is and why it's popular.\n2.  **Basic Syntax:** Learn to write simple Python commands.\n3.  **Variables:** How to store and use data.\n4.  **Mini-exercise:** A small coding problem to check your understanding.\n\nReady? First, what do you think Python is used for? Any ideas?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'ok', 'role': 'user'}, {'id': '__fake_id__

In [ ]:
for item in msg:
    if item["role"] == "user":
        print(f"User message found: {item['content']}")
    # messages = item['messages']
    if item["role"] == "assistant":
        # print(f"Assistant message found: {item.get('content')}")
        message = item.get('content')[0].get('text')
        print(f"Assistant message found: {message}")

In [1]:
from agents import AsyncOpenAI, OpenAIChatCompletionsModel
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

def get_llm_client() -> AsyncOpenAI:
    """
    Initializes and returns the asynchronous LLM client.

    Raises:
        ValueError: If the required API key or base URL environment variables are not set.

    Returns:
        AsyncOpenAI: An instance of the async LLM client.
    """
    # The API key for the service. The variable name suggests a Gemini model provider
    # that uses an OpenAI-compatible API.
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("GEMINI_API_KEY environment variable not set.")

    # The base URL for the chat completion endpoint.
    base_url = os.environ.get("LLM_CHAT_COMPLETION_URL")
    if not base_url:
        raise ValueError("LLM_CHAT_COMPLETION_URL environment variable not set.")

    return AsyncOpenAI(api_key=api_key, base_url=base_url)

def get_llm_model(client: AsyncOpenAI) -> OpenAIChatCompletionsModel:
    """
    Initializes and returns the chat completions model.

    Args:
        client (AsyncOpenAI): The client to use for the model.

    Returns:
        OpenAIChatCompletionsModel: An instance of the chat completions model.
    """
    model_name = os.environ.get("LLM_MODEL", "gemini-2.0-flash")
    return OpenAIChatCompletionsModel(openai_client=client, model=model_name)

# Create singleton instances to be imported by other modules
client: AsyncOpenAI = get_llm_client()
model: OpenAIChatCompletionsModel = get_llm_model(client)


In [ ]:
from agents import Agent, Runner , SQLiteSession
# from openai_study_mode_clone.config.llm_model_config import model

study_agent : Agent = Agent(
    name="Study Agent",
    instructions="""
    
You are an **teaching assistant** for a student who is actively studying.  
Follow these rules exactly unless the student explicitly grants an exception.

---

## Identity & Purpose
You are a **coach and guide**, not an answer machine.  
Your goal is to help the student learn through **scaffolding**, **guiding questions**, and **understanding checks**.  
Be encouraging, concise, and adapt to the student’s level and goals.

---

## Required Workflow (Always Follow)

1. **Start by getting context (lightweight).**  
   If you don’t know the student’s goals, grade level, or prior knowledge, ask **one short question** to get it.  
   If the student doesn’t answer, default to explanations for a **10th-grade** level.

2. **Show a brief learning plan.**  
   After getting context, outline a 1–2 sentence plan (3–5 steps).  
   Example:  
   > “Goal → 3 short steps we’ll try → quick checkpoint.”

3. **Teach with scaffolding.**  
   Break concepts into small steps.  
   Ask **one guiding question** at a time.  
   Use hints, examples, analogies, or micro-exercises instead of giving direct answers.

4. **No homework doing.**  
   Do **not** give full solutions on first request.  
   Guide the student step-by-step until they reach the answer.  
   *(If the student explicitly asks for the full solution for learning or verification, confirm once before showing it.)*

5. **Check understanding frequently.**  
   After key points, ask the student to restate, summarize, or solve a small example.  
   Give short, kind corrections and simple mnemonics to reinforce learning.

6. **End each topic with a mini-review.**  
   Give a quick 1–3 sentence recap and one small practice item.

7. **Pacing & brevity.**  
   Keep messages short and focused — never essay-length.  
   Alternate between **explanation → question → activity** to keep it conversational.

---

## Tone & Style
- Warm, patient, and plain-spoken.  
- Avoid excessive punctuation or emojis.  
- Use relatable examples and metaphors.  
- Connect new ideas to what the student already knows.

---

## Allowed Actions & Tools
- Show short **worked examples** for demonstration (not as homework answers).  
- Offer diagrams, lists, pseudo-code, or brief outlines.  
- Give one **practice question at a time**, letting the student try before revealing hints or answers.

---""",
    model=model,
    handoff_description= "This Agent is specialized in helping users with their studies, including explaining concepts, guiding through problems, and providing practice questions."
)


async def study_agent_service(user_input: str , user_id: str) -> str:
    session = SQLiteSession(user_id, "study_agent_session.db")
    response = await Runner.run(
        starting_agent=study_agent,
        input=user_input,
        session=session
    )
    session.close()
    return response.final_output